In [9]:
from collections import defaultdict
from itertools import combinations, product
import numpy as np

---

In [2]:
def binvec (a, n):
    """take a binary vector a of length n in its decimal representation and return its binary represenation"""
    return format(a, '0'+str(n)+'b')

def vector_weight (vector):
    """take a vector of length n and return its weight
    the weight of a vector is the number of nonzero entries in the vector
    """
    return sum([1 for i in vector if i != '0'])

def vecadd (a, b, n, q=2):
    """take two vectors a and b of length n and return their sum (mod q)"""
    return ''.join([str((int(a[i]) + int(b[i])) % q) for i in range(n)])
    #return format(a^b, '0'+str(n)+'b') for binary vectors only
    
def vecsadd (vectors, n, q=2):
    """take a set of vectors as a list of strings of ints of length n
    and return their sum (mod q) as a string of ints"""
    
    # verify that all vectors are length n
    for vector in vectors:
        assert len(vector) == n, 'vectors are assumed to be of equal length'
        
    
    positions = defaultdict(list)
    for vector in vectors:
        for i in range(n ):
            positions[i].append(vector[i])
    summ = []
    for i, j in positions.items():
        summ.append(np.mod(np.array(j).astype(int).sum(), q))
        
    return ''.join([str(i) for i in summ])
    
def vecinv (a, n):
    return format(a^(2**n-1), '0'+str(n)+'b')

def v (n, q=2):
    """construct a linear space of dimension n over a finite field of order q"""
    return [''.join(str(pos) for pos in comb) for comb in list(product(range(q), repeat=n))]

def num_lin_sub_k (n, k, q):
    """Compute the number of k-dim linear subspaces of a linear space of dim n over a finite field of order q"""
    return np.prod([q**n-q**i for i in range(k)])/np.prod([q**k-q**i for i in range(k)])

def generate_scalar_multiples (vector, n, q):
    """Given a vector of length n over q symbols, get its scalar multiples"""
    scalar_multiples = []
    for scalar in range(q):
        scalar_multiple = vector
        for i in range(scalar):
            scalar_multiple = vecadd(scalar_multiple, vector, n, q)
        scalar_multiples.append(scalar_multiple)
    return scalar_multiples

def span (vectors=['1000','0100'], n=4, q=2):
    """
    get the span of a set of vectors in V(n, q) over GF(q)
    """
    if '0'*n in vectors:
        vectors.remove('0'*n)
    qsm = []
    for veck in vectors:
        qsm.append(generate_scalar_multiples(veck, n, q))

    return set([vecsadd(comb, n, q) for comb in product(*[qsm[i] for i in range(len(qsm))])])

def gen_dis_bases (space, k):
    if k == 0:
        return [{0}], 0
    
    bases = list(combinations(space[1:], k))
    bases = list(set(i) for i in bases)
    #print(len(bases))
    #for i in bases:
    #    print(i)
            
    spans = defaultdict(list)
    while bases:
        basis, bases = bases[0], bases[1:]
        
        # check if the set of vectors is linearly independent
        if len(span(basis, n, q)) != q**k:
            continue
        
        s = span(basis, n, q)
        tot = ''
        for i in sorted(s):
            tot += i
        spans[tot].append(basis)
        
    return len(spans), [len(reprs) for spn, reprs in spans.items()], spans

def get_basis (space, k):
    num, l, bases = gen_dis_bases(space, k)
    return np.random.choice(bases[np.random.choice(np.array([spn for spn, reprs in bases.items()]))])

def gen_lin_code (basis, n, q):
    return span(basis, n, q)

In [171]:
"""
n = 8

a = binvec(31, n)
a_inv = vecinv(31, n)
b = binvec(64, n)

print(a)
print(a_inv)
print(b)
print(vecadd(a, b, n))
"""

00011111
11100000
01000000
01011111


---

### Construct $V(n, q)$ over $GF(q)$ and compute the scalar multiples of a vector $\{\lambda\mathbf{v}\,\vert\,\mathbf{v}\in V(n, q), \forall \lambda\in GF(q)\}$

$| V(n, q) | = q^n$

In [3]:
n = 5           # specify the dimension of V(n, q)
q = 2           # specify the order of GF(q)
space = v(n, q) # construct V(n, q) over GF(q)

print('V(' + str(n) + ', ' + str(q) + ')')
print()
print('{}\t{}\t{}'.format(str(n**q) + ' vectors', 'weight', str(q) + ' scalar multiples per vector'))
print()
for i, vector in enumerate(space):
    print('{}\t{}\t{}\t{}'.format(i, vector, vector_weight(vector), generate_scalar_multiples(vector, n, q)))

V(5, 2)

25 vectors	weight	2 scalar multiples per vector

0	00000	0	['00000', '00000']
1	00001	1	['00001', '00000']
2	00010	1	['00010', '00000']
3	00011	2	['00011', '00000']
4	00100	1	['00100', '00000']
5	00101	2	['00101', '00000']
6	00110	2	['00110', '00000']
7	00111	3	['00111', '00000']
8	01000	1	['01000', '00000']
9	01001	2	['01001', '00000']
10	01010	2	['01010', '00000']
11	01011	3	['01011', '00000']
12	01100	2	['01100', '00000']
13	01101	3	['01101', '00000']
14	01110	3	['01110', '00000']
15	01111	4	['01111', '00000']
16	10000	1	['10000', '00000']
17	10001	2	['10001', '00000']
18	10010	2	['10010', '00000']
19	10011	3	['10011', '00000']
20	10100	2	['10100', '00000']
21	10101	3	['10101', '00000']
22	10110	3	['10110', '00000']
23	10111	4	['10111', '00000']
24	11000	2	['11000', '00000']
25	11001	3	['11001', '00000']
26	11010	3	['11010', '00000']
27	11011	4	['11011', '00000']
28	11100	3	['11100', '00000']
29	11101	4	['11101', '00000']
30	11110	4	['11110', '00000']
31	11111	5	['11111', '

In [4]:
# show that the sum of any two vectors in the linear space is also in the linear space

a = np.random.choice(space) # choose a random vector in V(n, q)
b = np.random.choice(space) # choose a random vector in V(n, q)
c = vecadd(a, b, n, q)      # add them
print(a)
print(b)
print(c)
assert c in space, 'Error: the sum of the vectors both of which are in V(n, q) is not in V(n, q)'

10011
11100
01111


---

### Compute the sum $\mathbf{v_1} + ... + \mathbf{v_k}$ of a set of vectors $\{\mathbf{v_1}, ..., \mathbf{v_k}\} \in V(n, q)$

In [6]:
num_vecs = np.random.randint(1, 5)
sum_vecs = np.random.choice(space, size=num_vecs, replace=True)
summation = vecsadd(sum_vecs, n, q)

print('Summing {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(sum_vecs)))
print('Sum\t\t\t{}'.format(summation))

Summing 3 vectors	11100 + 00101 + 11010
Sum			00011


### Get the span $span\{\mathbf{v_1}, ..., \mathbf{v_k}\}$ of a set of vectors $\{\mathbf{v_1}, ..., \mathbf{v_k}\} \in V(n, q)$

In [7]:
span_vecs = list(np.random.choice(space[1:], size=num_vecs, replace=False))
spn = span(span_vecs, n, q)

print('Spanning {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(span_vecs)))
print('Span {} vector{}\t\t{}'.format(len(spn), 's' if num_vecs != 1 else '', spn))

Spanning 3 vectors	01001 + 11101 + 00001
Span 8 vectors		{'10101', '01001', '11100', '10100', '00001', '01000', '00000', '11101'}


---

### Compute the number of distinct k-dim linear subspaces

The number of distinct linear subspaces of dim $k$ of a linear space of dim $n$ is $\frac{(q^n - q^0)(q^n - q^1)...(q^n - q^{k - 1})}{(q^k - q^0)(q^k - q^1)...(q^k - q^{k - 1})}$

In [10]:
# Compute the total number of linear subspaces of a linear space of dim n
# i.e., compute the number of k-dim linear subspaces for k = 1, ..., n
# the "0-dim" linear subspace is the trivial subspace {0}

print('{}\t{}\t{}\t{}'.format('k', 'q^k', 'bases', 'q^n_choose_k'))
print()
for k in range(0, n + 1):
    print('{}\t{}\t{}\t{}'.format(k, q**k, int(num_lin_sub_k(n, k, q)), len(list(combinations(range(q**n), k)))))

k	q^k	bases	q^n_choose_k

0	1	1	1
1	2	31	32
2	4	155	496
3	8	155	4960
4	16	31	35960
5	32	1	201376


---

### Compute distinct k-dim bases and their representations

In [60]:
bases = gen_dis_bases(space, k=3)

---

### Get a k-dim basis

In [54]:
basis = get_basis(space, k=3)
basis

{'00001', '10000', '11111'}

---

### Generate a linear [n, k]-code

In [55]:
for i in sorted(gen_lin_code(basis, n, q)):
    print(i)

00000
00001
01110
01111
10000
10001
11110
11111


### Generate a linear [n, k]-code with a generator matrix

In [56]:
messages = np.array([list(i) for i in v(3, q)], dtype=int)
messages

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [57]:
generator = np.array(list(tuple(i) for i in basis), dtype=int)
generator

array([[1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1]])

In [61]:
np.dot(messages, generator) % q

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 1, 0]])